In [1]:
import numpy as np

# RNN model

### Utility functions

In [17]:
def softmax(x):
  e_x = np.exp(x - np.max(x))
  
  return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [7]:
def rnn_cell_forward(timestep_t, a_prev, parameters):
  """
  Implements a single forward step of an RNN cell

  Args:
    timestep_t: (numpy array) input data at timestep "t"
    a_prev: (numpy array) hidden state at timestep "t-1"
    parameters: (dict) containing:
      Wax: (numpy array) weight matrix multiplying the input
      Waa: (numpy array) weight matrix multiplying the hidden state
      Wya: (numpy array) weight matrix relating the hiden state
        to the output
      ba: (numpy array) bias
      by: (numpy array) bias relating the hidden state to the output

  Returns:
    a_next: (numpy array) next hidden state
    yt_pred: (numpy array) prediction at timestep "t"
    cache: (tuple) with values needed for the backward pass
      it contains (a_nex, a_prev, timestep_t, parameters)
  """

  Wax = parameters['Wax']
  Waa = parameters['Waa']
  Wya = parameters['Wya']
  ba = parameters['ba']
  by = parameters['by']

  # Compute next activation state
  a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, timestep_t) + ba)

  # Compute output of the current cell
  yt_pred = softmax(np.dot(Wya, a_next) + by)

  cache = (a_next, a_prev, timestep_t, parameters)

  return a_next, yt_pred, cache

# RNN forward pass

In [11]:
def rnn_forward(x, act0, parameters):
  """
  Implements the forward propagation of a recurrent neural
  network

  Args:
    x: (numpy array) input data for every timestep
    act0: (numpy array) initial hidden state
    parameters: (dict) containing:
      Wax: (numpy array) weight matrix multiplying the input
      Waa: (numpy array) weight matrix multiplying the hidden state
      Wya: (numpy array) weight matrix relating the hiden state
        to the output
      ba: (numpy array) bias
      by: (numpy array) bias relating the hidden state to the output

  Returns:
    activations: (numpy array) hidden states for every timestep
    y_pred: (numpy array) predictions for every timestep
    caches: (tuple) with values needed for the backward pass
      it contains the list of caches and x
  """

  caches = []

  # get dimensions from shapes of x and Wy
  n_x, m, T_x = x.shape
  n_y, n_a = parameters['Wya'].shape

  # initialize 'a' and 'y' with zeros
  activations = np.zeros((n_a, m, T_x))
  y_pred = np.zeros((n_y, m, T_x))

  # initialize a_next
  a_next = act0

  for timestep in range(T_x):
    # update next hidden state, compute the prediction
    a_next, yt_pred, cache = rnn_cell_forward(x[:, :, timestep], a_next, parameters)

    # save the value of the new 'next' hidden state and the prediction
    activations[:, :, timestep] = a_next
    y_pred[:, :, timestep] = yt_pred
    caches.append(cache)

  caches = (caches, x)

  return activations, y_pred, caches

# Backprop in a RNN


In [13]:
def rnn_cell_backward(da_next, cache):
  """
  Implements the backward pass for a single step in the RNN-cell

  Args:
    da_next: (numpy array) gradient loss with respect to next hidden state
    cache: (dict) contains outputs of the rnn_cell_forward()

  Returns:
    gradients: (dict) containing:
      dx: (numpy array)
      da_prev: (numpy array) gradients of previous hidden state
      dWax: (numpy array) gradients of input to hidden weights
      dWaa: (numpy array) gradients of hidden to hidden weights
      dba: (numpy array) gradients of bias vector
  """

  a_next, a_prev, timestep_t, parameters = cache

  Wax = parameters['Wax']
  Waa = parameters['Waa']
  Wya = parameters['Wya']
  ba = parameters['ba']
  by = parameters['by']

  # compute the gradient of tanh with respect to a_next
  dtanh = (1 - a_next**2) * da_next

  # compute the gradient of the loss with respect to Wax
  dxt = np.dot(Wax.T, dtanh)
  dWax = np.dot(dtanh, timestep_t.T)

  # Compute the gradient with respect to Waa
  da_prev = np.dot(Waa.T, dtanh)
  dWaa = np.dot(dtanh, a_prev.T)

  # compute the gradient with respect to b
  dba = np.sum(dtanh, 1, keepdims=True)

  # store gradients
  gradients = {'dx': dxt,
               'dWax': dWax,
               'da_prev': da_prev,
               'dWaa': dWaa,
               'dba': dba}

  return gradients

In [15]:
def rnn_backward(da, caches):
  """
  Implements the backward pass for a RNN over an entire sequence
  of input data

  Args:
    da: upstream gradient of all hidden states
    caches: (tuple) containing information from the forward pass

  Returns:
    gradients: (dict) containing:
      dx: gradients of the input data
      da0: gradients of the initial hidden state
      dWax: gradients of the input's weights matrix
      dWaa: gradients of the hidden state's weights matrix
      dba: gradients of the bias
  """

  caches, x = caches
  a1, a0, x1, parameters = caches[0]

  # get dimensions
  n_a, m, T_x = da.shape
  n_x, m = x1.shape

  # initialize the gradients with the right sizes
  dx = np.zeros((n_x, m, T_x))
  dWax = np.zeros((n_a, n_x))
  dWaa = np.zeros((n_a, n_a))
  dba = np.zeros((n_a, 1))
  da0 = np.zeros((n_a, m))
  da_prevt = np.zeros((n_a, m))

  for timestep in reversed(range(T_x)):
    # compute gradients in current timestep
    gradients = rnn_cell_backward(da[:, :, timestep] + da_prevt, caches[timestep])

    # get derivatives from gradients
    dxt = gradients['dxt'] 
    da_prevt = gradients['da_prev']
    dWaxt = gradients['dWax']
    dWaat = gradients['dWaa']
    dbat = gradients['dba']

    # increment the global derivatives
    dx[:, :, timestep] = dxt
    dWax += dWaxt
    dWaa += dWaat
    dba += dbat

  # set da0 to the gradient of "a" which has been backpropagated thorugh
  # all the timesteps
  da0 = da_prevt

  gradients = {'dx': dx,
               'da0': da0,
               'dWax': dWax,
               'dWaa': dWaa,
               'dba': dba}

  return gradients  


# LSTM model


In [35]:
def lstm_cell_forward(timestep_t, a_prev, c_prev, parameters):
  """
  Implements a single forward step of a LSTM cell

  Args:
    timestep_t: (numpy array) input data at timestep 't'
    a_prev: (numpy array) hidden state at timestep 't-1'
    c_prev: (numpy array) memory state at timestep 't-1'
    parameters: (dict) containing:
      Wf: weight matrix of the forget gate
      bf: bias of the forget gate
      Wu: weight matrix of the update gate
      bu: bias of the update gate
      Wc: weight matrix of the first "tanh"
      bc: bias of the first "tanh"
      Wo: weight matrix of the output gate
      bo: bias of the output gate
      Wy: weight matrix relating the hidden state to the output
      by: bias relating the hidden state to the output

  Returns: 
    a_next: (numpy array) next hidden state
    c_next: (numpy array) next memory state
    yt_pred: (numpy array) prediction at timestep 't'
    cache: (tuple) with values needed for the backprop
  """
  Wf = parameters["Wf"]
  bf = parameters["bf"]
  Wu = parameters["Wu"]
  bu = parameters["bu"]
  Wc = parameters["Wc"]
  bc = parameters["bc"]
  Wo = parameters["Wo"]
  bo = parameters["bo"]
  Wy = parameters["Wy"]
  by = parameters["by"]

  # get rigth dimensions
  n_x, m = timestep_t.shape
  n_y, n_a = Wy.shape

  # concatenate a_prev and timestep_t
  concat = np.zeros((n_a + n_x, m))
  concat[:n_a, :] = a_prev
  concat[n_a:, :] = timestep_t

  # compute gates
  for_t = sigmoid(np.dot(Wf, concat) + bf)
  up_t = sigmoid(np.dot(Wu, concat) + bu)
  cct = np.tanh(np.dot(Wc, concat) + bc)
  c_next = for_t * c_prev + up_t * cct
  out_t = sigmoid(np.dot(Wo, concat) + bo)
  a_next = out_t * np.tanh(c_next)

  # compute prediction of the LSTM cell
  yt_pred = softmax(np.dot(Wy, a_next) + by)

  cache = (a_next, c_next, a_prev, c_prev, for_t, up_t, cct, out_t, timestep_t, parameters)

  return a_next, c_next, yt_pred, cache


In [36]:
def lstm_forward(x, a0, parameters):
  """
  Implements the forward pass of the RNN using an LSTM cell

  Args:
    x: input data for every timestep
    a0: initial hidden state
    parameters: (dict) containing:
      Wf: weight matrix of the forget gate
      bf: bias of the forget gate
      Wu: weight matrix of the update gate
      bu: bias of the update gate
      Wc: weight matrix of the first "tanh"
      bc: bias of the first "tanh"
      Wo: weight matrix of the output gate
      bo: bias of the output gate
      Wy: weight matrix relating the hidden state to the output
      by: bias relating the hidden state to the output

  Returns:
    activations: (numpy array) hidden states for every timestep
    y: (numpy array) predictions for every timestep
    c: (numpy array) cell state for every timestep
    caches: tuple with values needed for the backprop
  """
  caches = []

  # get right dimensions
  n_x, m, T_x = x.shape
  n_y, n_a = parameters['Wy'].shape

  # initialize
  activations = np.zeros((n_a, m, T_x))
  c = activations
  y = np.zeros((n_y, m, T_x))
  a_next = a0
  c_next = np.zeros(a_next.shape)

  for timestep in range(T_x):
    # update next hidden state, next meomry state, and compute predictions
    a_next, c_next, yt, cache = lstm_cell_forward(x[:, :, timestep], a_next, c_next, parameters)

    # save the value of the new "next" hidden state, the prediction
    # and the next cell state
    activations[:, :, timestep] = a_next
    y[:, :, timestep] = yt
    c[:, :, timestep] = c_next
    caches.append(cache)

  caches = (caches, x)

  return activations, y, c, caches

# Backpropagation for LSTM

In [39]:
def lstm_cell_backward(da_next, dc_next, cache):
  """
  Implements a single timestep backprop for the lstm cell

  Args:
    da_next: gradients of next hidden state
    dc_next: gradients of next cell state
    cache: information from the forward pass

  Returns:
    gradients: (dict) containing:
      dxt: gradient of input data at timestep t
      da_prev: gradient of the previous hidden state
      dc_prev: gradient  of the previous memory state
      dWf: gradient of the weight matrix of the forget gate
      dWu: gradient of the weight matrix of the update gate
      dWc: gradient of the weight matrix of the memory gate
      dWo: gradient of the weight matrix of the output gate
      dbf: gradient of the baises of the fortget gate
      dbu: gradient of the biases of the update gate
      dbc: gradient of the biases of the memory gate
      dbo: gradient of the biases of the output gate
  """

  a_next, c_next, a_prev, c_prev, for_t, up_t, cct, out_t, xt, parameters = cache

  # get right dimensions
  n_x, m = xt.shape
  n_a, m = a_next.shape

  # Compute gates related derivatives
  dot = da_next * np.tanh(c_next) * out_t * (1 - out_t)
  dcct = (dc_next * up_t + out_t * (1 - np.square(np.tanh(c_next))) * up_t * da_next) * (1 - np.square(cct))
  dut = (dc_next * cct + out_t * (1 - np.square(np.tanh(c_next))) * cct * da_next) * up_t * (1 - up_t)
  dft = (dc_next * c_prev + out_t * (1 - np.square(np.tanh(c_next))) * c_prev * da_next) * for_t * (1 - for_t)

  concat = np.concatenate((a_prev, xt), axis=0)

  # compute parameters related derivatives
  dWf = np.dot(dft, concat.T)
  dWu = np.dot(dut, concat.T)
  dWc = np.dot(dcct, concat.T)
  dWo = np.dot(dot, concat.T)
  dbf = np.sum(dft, axis=1, keepdims=True)
  dbu = np.sum(dut, axis=1, keepdims=True)
  dbc = np.sum(dcct, axis=1, keepdims=True)
  dbo = np.sum(dot, axis=1, keepdims=True)

  # Compute derivatives: previous hidden state, previous memory state and input
  Wf = parameters['Wf']
  Wu = parameters['Wu']
  Wc = parameters['Wc']
  Wo = parameters['Wo']
  da_prev = np.dot(Wf[:, :n_a].T, dft) + np.dot(Wu[:, :n_a].T, dut) + np.dot(Wc[:, :n_a].T, dcct) + np.dot(Wo[:, :n_a].T, dot)
  dc_prev = dc_next * for_t + out_t * (1 - np.square(np.tanh(c_next))) * for_t * da_next
  dxt = np.dot(Wf[:, n_a:].T, dft) + np.dot(Wu[:, n_a:].T, dut) + np.dot(Wc[:, n_a:].T, dcct) + np.dot(Wo[:, n_a:].T, dot)

  gradients = {'dxt': dxt,
               'da_prev': da_prev,
               'dc_prev': dc_prev,
               'dWf': dWf,
               'dbf': dbf,
               'dWu': dWu,
               'dbu': dbu,
               'dWc': dWc,
               'dbc': dbc,
               'dWo': dWo,
               'dbo': dbo}

  return gradients

In [53]:
def lstm_backward(da, caches):
  """
  Implements the backward pass for the whole sequence 
  of the RNN with lstm cell

  Args:
    da: gradients of the hidden states
    cache: information from the forward pass
  
  Returns:
    gradients: (dict) containing:
      dx: gradient of inputs
      da0: gradient of the previous hidden state
      dWf: gradient of the weight matrix of the forget gate
      dWu: gradient of the weight matrix of the update gate
      dWc: gradient of the weight matrix of the memory gate
      dWo: gradient of the weight matrix of the output gate
      dbf: gradient of the baises of the fortget gate
      dbu: gradient of the biases of the update gate
      dbc: gradient of the biases of the memory gate
      dbo: gradient of the biases of the output gate
  """
  caches, x = caches
  a1, c1, a0, c0, f1, u1, cc1, o1, x1, parameters = caches[0]
  
  # get right dimensions
  n_a, m, T_x = da.shape
  n_x, m = x1.shape

  # initialize the gradients
  dx = np.zeros((n_x, m, T_x))
  da0 = np.zeros((n_a, m))
  da_prevt = np.zeros(da0.shape)
  dc_prevt = np.zeros(da0.shape)
  dWf = np.zeros((n_a, n_a + n_x))
  dWu = np.zeros(dWf.shape)
  dWc = np.zeros(dWf.shape)
  dWo = np.zeros(dWf.shape)
  dbf = np.zeros((n_a, 1))
  dbu = np.zeros(dbf.shape)
  dbc = np.zeros(dbf.shape)
  dbo = np.zeros(dbf.shape)

  for timestep in reversed(range(T_x)):
    # Compute all gradients
    gradients = lstm_cell_backward(da[:, :, timestep], dc_prevt, caches[timestep])

    # Store or add the gradient to the parameters
    dx[:, :, timestep] = gradients['dxt']
    dWf += gradients['dWf']
    dWu += gradients['dWu']
    dWc += gradients['dWc']
    dWo += gradients['dWo']
    dbf += gradients['dbf']
    dbu += gradients['dbu']
    dbc += gradients['dbc']
    dbo += gradients['dbo']

  # set the first activation's gradient to the backpropagated gradient da_prev
  da0 = gradients['da_prev']

  gradients = {'dx': dx,
               'da0': da0,
               'dWf': dWf,
               'dWu': dWu,
               'dWc': dWc,
               'dWo': dWo,
               'dbf': dbf,
               'dbu': dbu,
               'dbc': dbc,
               'dbo': dbo}

  return gradients